In [15]:
import os
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tqdm import tqdm  
from torchvision.models import alexnet, AlexNet_Weights
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader

Dataset Processing and Parameters

In [16]:
data_dir = "../mri-images/augmented-images-v3"
batch_size = 32
num_epochs = 30
learning_rate = 0.001  # Learning rate parameter


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
# Transformations: replicate grayscale channels to match AlexNet's input requirements (3 channels)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Replicate grayscale to 3 channels
    transforms.Resize((224, 224)),               # Resize to AlexNet input size
    transforms.ToTensor(),                       # Convert to tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize channels
])

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Prepare 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Results dictionary to store validation results for each fold
results = {}
results_train={}

# Perform cross-validation and store metrics for all folds
all_train_losses, all_val_losses = [], []
all_train_accuracies, all_val_accuracies = [], []

In [19]:
print(f"Number of classes: {len(dataset.classes)}")
print(f"Class names: {dataset.classes}")

Number of classes: 3
Class names: ['AD', 'CN', 'MCI']


In [20]:
class_counts = Counter(dataset.targets)
print(f"Number of images in each class: {class_counts}")

Number of images in each class: Counter({2: 932, 0: 916, 1: 916})


In [ ]:
# early stopping criteria
patience = 5  # Number of epochs to wait for improvement

screen_width = 80
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    # Create the fold header with "--" padding
    fold_header = f" Fold {fold_idx+1}/{kf.n_splits} "
    padding = (screen_width - len(fold_header)) // 2
    print(f"{'-' * padding}{fold_header}{'-' * padding}")
    
    # Split dataset into train and validation sets based on indices
    train_subset = torch.utils.data.Subset(dataset, train_idx)
    val_subset = torch.utils.data.Subset(dataset, val_idx)
    
    train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    
    # Load AlexNet with updated pre-trained weights
    # Instantiate a new AlexNet model for each fold
    weights = AlexNet_Weights.DEFAULT 
    model = alexnet(weights=weights)
    model.classifier[6] = nn.Linear(4096, len(dataset.classes))  # Modify final layer to match number of classes
    model = model.to(device)

    # Move the model to the GPU
    model = model.to(device)
        
    criterion = nn.CrossEntropyLoss()
    
    # Use SGD optimizer with momentum
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4)

    # Add a ReduceLROnPlateau scheduler
    # scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

    # Early stopping parameters
    best_val_loss = float('inf')  # Initialize best validation loss
    epochs_without_improvement = 0  # Counter for epochs without improvement

    # Store metrics for plotting later
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        # Training phase with tqdm progress bar
        model.train()
        running_loss, correct_preds, total_samples = 0.0, 0, 0
        
        train_loader_tqdm = tqdm(train_loader, desc="Training", leave=False)
        for inputs, labels in train_loader_tqdm:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += (preds == labels).sum().item()
            total_samples += labels.size(0)
        
        epoch_train_loss = running_loss / total_samples
        epoch_train_acc = correct_preds / total_samples
        
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        print(f"Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {epoch_train_acc:.4f}")
      
        # Validation phase with tqdm progress bar
        model.eval()
        running_loss, correct_preds, total_samples = 0.0, 0, 0
        
        val_loader_tqdm = tqdm(val_loader, desc="Validation", leave=False)
        with torch.no_grad():
            for inputs, labels in val_loader_tqdm:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct_preds += (preds == labels).sum().item()
                total_samples += labels.size(0)
        
        epoch_val_loss = running_loss / total_samples
        epoch_val_acc = correct_preds / total_samples
        
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        print(f"Val Loss: {epoch_val_loss:.4f}, Val Accuracy: {epoch_val_acc:.4f}")
        # scheduler.step(epoch_val_loss)

        # Log the updated learning rate
        # current_lr = scheduler.get_last_lr()
        # print(f"Epoch {epoch+1}: Current Learning Rate: {current_lr}")

        # Early stopping logic
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            epochs_without_improvement = 0  # Reset counter
            print("Validation loss improved. ")
            # print("Saving Model...")
            # torch.save(model.state_dict(), f"best_model_fold_{fold_idx+1}.pth")  # Save the best model
        else:
            epochs_without_improvement += 1
            print(f"No improvement in validation loss for {epochs_without_improvement} epoch(s).")

        if epochs_without_improvement >= patience:
            print("Early stopping triggered.")
            break

    all_train_losses.append(train_losses)
    all_val_losses.append(val_losses)
    all_train_accuracies.append(train_accuracies)
    all_val_accuracies.append(val_accuracies)

    # Store validation accuracy of this fold in the results dictionary
    results[f"Fold {fold_idx+1}"] = epoch_val_acc
    results_train[f"Fold {fold_idx+1}"] = epoch_train_acc

In [22]:
# Calculate average accuracy across all folds
average_accuracy = np.mean(list(results.values()))

# Print results for each fold and average accuracy
print("\nValidation Results:")
for fold_name, accuracy in results.items():
    print(f"{fold_name}: {accuracy:.4f}")
print(f"Average Validation Accuracy: {average_accuracy:.4f}")


Validation Results:
Fold 1: 0.6600
Fold 2: 0.6456
Fold 3: 0.6239
Fold 4: 0.5407
Fold 5: 0.6504
Average Validation Accuracy: 0.6241


In [23]:
# Calculate average accuracy across all folds
average_train_accuracy = np.mean(list(results_train.values()))

# Print results for each fold and average accuracy
print("\nTrain Results:")
for fold_name, accuracy in results_train.items():
    print(f"{fold_name}: {accuracy:.4f}")
print(f"Average Train Accuracy: {average_train_accuracy:.4f}")


Train Results:
Fold 1: 0.8919
Fold 2: 0.9611
Fold 3: 0.9711
Fold 4: 0.7386
Fold 5: 0.9358
Average Train Accuracy: 0.8997
